In [1]:
#Functions and Imports 

import pandas as pd
import duckdb
import openpyxl
from openpyxl.utils import get_column_letter
import math
import numpy as np

# Setting up SQL syntax in Python 
#conn = duckdb.connect("SampleDB",read_only=False)

# Ensure pandas ExcelWriter uses a different engine if openpyxl is not available
try:
    from openpyxl import Workbook
except ModuleNotFoundError:
    
    excel_engine = 'xlsxwriter'  
else:
    excel_engine = 'openpyxl'




#Function that gets total cost from dataframe
#Parameters: dataframe
#Returns: float 
def costValue(df):
    df = df.iloc[:,infoColumn:numEndColumn].sum()
    df = pd.DataFrame(df)
    for i in range(len(df)):
        if i%2 != 0:
            df.iloc[i,0] = df.iloc[i,0] * -1
    df = df.sum()        
    return df    

#Function that checks if newfile csv has the correct datatypes to run 
#Parameters: dataframe 
#Returns: adjusted dataframe with float dtype columns
def dataType(df):
    dft = df.iloc[:,infoColumn:numEndColumn]
    dft = pd.DataFrame(dft)
    for i in range(len(dft.columns)):
        if dft[dft.columns[i]].dtype != float:
            #print(dft.columns[i])
            df[dft.columns[i]] = dft[dft.columns[i]].str.replace(',','').astype(float)
    return df    
        


#Function that gets dataframe costValue 
#Parameters: array of dataframes and float 
#Returns: returns the summed dataframes total cost of the dataframes
#Notes: Prints error if summed total cost of dataframes does not match totalcost float  
def checkValue(dfs,totalCost):
    costValueTotal = 0 
    for i in range(len(dfs)):
        costValueTotal = costValueTotal + costValue(dfs[i])
    costValueTotal = costValueTotal * 100 
    totalCost = totalCost * 100
    checker = (totalCost - costValueTotal).astype(int)
    if checker.all() != 0:
        print("Error total cost was changed")
    return (costValueTotal / 100)   


#Variables for functions (Update if columns were added to UPS file)
infoColumn = 33
numEndColumn = 65
billedWeightColmn = 28

    

In [19]:
#importing two files: Res1 and ups csv

#importing 
UPSCSV = pd.read_csv("Invoice_00000399CF_110125.csv", encoding='latin',low_memory=False, header=None)
UPSCSV = UPSCSV.fillna('')
res1 = pd.read_excel('res1 adjusted 1.xlsx')

#Searching common tracking numbers in UPSCSV file 
UPSCSV_Res1 = UPSCSV[UPSCSV[13].isin(res1['Tracking Number'])]

#Simplifying ups datafram 
UPSCSV_Res1 = pd.DataFrame(UPSCSV_Res1.iloc[:,[20,43,45,51,52]])

#Getting Charge Value column and adding it as new column 
sumTotal = UPSCSV_Res1.iloc[:,3] + UPSCSV_Res1.iloc[:,4]
UPSCSV_Res1['Charge Amount'] = sumTotal
UPSCSV_Res1


,20,43,45,51,52,Charge Amount
13012,1Z1FV6990395559755,FRT,Ground Commercial,88.17,36.01,124.18
13013,1Z1FV6990395559755,ACC,Addl. Handling weight,27.5,27.5,55.0
13014,1Z1FV6990395559755,ACC,Demand Surcharge Addl Handling,0.0,8.25,8.25
13015,1Z1FV6990395559755,FSC,Fuel Surcharge,27.78,10.17,37.95
13196,1Z1FV6990395559755,FRT,Residential Adjustment,-4.97,4.97,0.0
13197,1Z1FV6990395559755,ACC,Residential Surcharge Adjustment,3.9,2.2,6.1
13198,1Z1FV6990395559755,FSC,Fuel Surcharge Adjustment,0.44,1.01,1.45


In [23]:
#Getting Total Amount for Res1 file
#getting grouped by tracking number total charge  
groupedRes1 = UPSCSV_Res1.groupby(20).sum()
groupedRes1 = pd.DataFrame(groupedRes1)

#Getting the cost value for each shipment in Res1 file
res2 = pd.DataFrame(res1.iloc[:,[4]+list(range(infoColumn,numEndColumn))])

#adjusting incentive columns to be negative values
for i in range(len(res2.columns)-1):
    if (i+1)%2 == 0:
        res2.iloc[:,i+1] = res2.iloc[:,i+1] * -1
#Creating a new column to validate charges in UPS dataframe and res 
res2["Total Amount"] = res2.iloc[:,1:len(res2.columns)].sum(axis=1)

#Simplifying dataframes 
simpleRes = res2[['Tracking Number','Total Amount']]
simpleGrouped = pd.DataFrame(groupedRes1.iloc[:,3])
simpleGrouped = simpleGrouped.reset_index(names=['Tracking Number'])


In [25]:
#Validating the Total Amount in both arrays
merged = pd.merge(simpleRes,simpleGrouped)

#adding a match column to compare values
merged['Match'] = (abs(merged.iloc[:,1]-merged.iloc[:,2]))


for i in range(len(merged)):
    if math.floor(merged.iloc[i,3]) != 0:
        print('Warning Mismatch: ' + str(merged.iloc[i,0]))

              

Warning Mismatch: 1ZA83J550321160778


In [31]:
#Displaying merged dataframe to compare total shipping charges (determine if CSV totals match audintel totals)
merged

,Tracking Number,Total Amount,52,Match
0,1Z1FV6990395559755,90.11,90.11,0.0
1,1Z1FV6990396375577,80.12,80.12,0.0
2,1Z1FV6990396738961,76.60,76.6,0.0
3,1Z4X459A4204799741,31.52,31.52,0.0
4,1Z4X459A4210604322,24.33,24.33,0.0
5,1Z4X459A4215211336,24.81,24.81,0.0
6,1Z7EW7860230845078,19.79,19.79,0.0
7,1Z84W64A0308207152,14.48,14.48,0.0
8,1Z84W64A0325902547,9.65,9.65,0.0
9,1Z84W64A0333588659,9.75,9.75,0.0


In [1046]:
#importing mapping (Mapping external sheet that is a manual sheet that maps "Name" to a column number 
mapping = pd.read_excel('UPS Mapping.xlsx')
mapping

,Code,Code2,Name,Incentive,Charge Diff,Mapping
0,FRT,3,Ground Commercial,14.38,6.11,Other
1,FSC,FSC,Fuel Surcharge,3.24,0.86,Other
2,INF,3,Ground Commercial,0.00,0.00,Other
3,FRT,RES,Residential Adjustment,-0.82,0.82,Other
4,ACC,RES,Residential Surcharge Adjustment,3.90,2.20,Other
...,...,...,...,...,...,...
184,FSC,FSC,Fuel Surcharge Adjustment,0.24,0.57,Other
185,FRT,RES,Residential Adjustment,-0.25,0.25,Other
186,ACC,RES,Residential Surcharge Adjustment,3.90,2.20,Other
187,ACC,DSR,Demand Surcharge - Residential Adjustment,0.30,0.10,Other


In [1058]:
#Assigning Mapping dataframe a mapping column.
mapping['Mapping'] = numEndColumn
mapping

,Code,Code2,Name,Incentive,Charge Diff,Mapping,Mapping
0,FRT,3,Ground Commercial,14.38,6.11,Other,65
1,FSC,FSC,Fuel Surcharge,3.24,0.86,Other,65
2,INF,3,Ground Commercial,0.00,0.00,Other,65
3,FRT,RES,Residential Adjustment,-0.82,0.82,Other,65
4,ACC,RES,Residential Surcharge Adjustment,3.90,2.20,Other,65
...,...,...,...,...,...,...,...
184,FSC,FSC,Fuel Surcharge Adjustment,0.24,0.57,Other,65
185,FRT,RES,Residential Adjustment,-0.25,0.25,Other,65
186,ACC,RES,Residential Surcharge Adjustment,3.90,2.20,Other,65
187,ACC,DSR,Demand Surcharge - Residential Adjustment,0.30,0.10,Other,65


In [1060]:
#creating a dict zip file to map dataframe 
mapDict = dict(zip(mapping['Name'],mapping['Mapping']))
mapDict

{'Ground Commercial': 65,
 'Fuel Surcharge': 65,
 'Residential Adjustment': 65,
 'Residential Surcharge Adjustment': 65,
 'Demand Surcharge - Residential Adjustment': 65,
 'Fuel Surcharge Adjustment': 65,
 '2nd Day Air Commercial': 65,
 'Delivery Area Surcharge': 65,
 'Delivery Area Surcharge Adjustment': 65,
 '3 Day Select Commercial': 65,
 'Delivery Confirmation Signature - Commercial': 65,
 'Demand Surcharge - Commercial': 65,
 'Address Correction 3 Day Select': 65,
 'Address Correction Fuel Surcharge': 65,
 'Delivery Confirmation Signature - Commercial Adjustment': 65,
 'Delivery Confirmation Signature Adjustment': 65,
 'Demand Surcharge - Commercial Adjustment': 65,
 'Address Correction Ground': 65,
 'Addl. Handling weight': 65,
 'Demand Surcharge Addl Handling': 65,
 'Shipping Charge Correction Ground': 65,
 'Shipping Charge Correction Fuel Surcharge': 65,
 'Delivery Area Surcharge - Extended': 65,
 'Delivery Area Surcharge - Extended Adjustment': 65}

In [1082]:
#Making Base Freight adjustment for Base charge & incentive

#Creating an empty dataframe for adjustments
newRes = res2.iloc[:,list(range(1,numEndColumn-infoColumn))] * 0
newRes['Tracking Number'] = res2['Tracking Number']

#Filtering for FRT only (Freight Base & Charges Adjustments)
UpsCsvFRT = UPSCSV_Res1[UPSCSV_Res1.iloc[:,1] == 'FRT']
#Getting total for FRT
groupedUpsCsvFRT = pd.DataFrame(UpsCsvFRT.groupby([20]).sum())

for i in range(len(newRes)):
    newRes.iloc[i,0] = groupedUpsCsvFRT.iloc[i,2] + groupedUpsCsvFRT.iloc[i,3]
    newRes.iloc[i,1] = groupedUpsCsvFRT.iloc[i,2]
    

In [ ]:
#Making remaning adjustments using Mapping dataframe

#Pending 




In [1084]:
#Dispalying updated dataframe after Base Freight & Charge incentives  
newRes

,Base Freight Charge,Base Incentive Charge,Additional Handling Package,Additional Handling Package Incentive,Additional Handling Weight,Additional Handling Weight Incentive,Additional Handling Dimensions,Additional Handling Dimensions Incentive,Adult Sign Req Freight,Adult Sign Req Incentive,...,Address Correction,Address Correction Incentive,Print Labels,Print Labels Incentive,Electronic Labels,Electronic Labels Incentive,Large Package Surcharge,Large Package Surcharge Incentive,Other Freight Charge,Tracking Number
0,124.18,83.20,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,0.0,1Z1FV6990395559755
1,97.63,65.41,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,0.0,1Z1FV6990396375577
2,88.30,59.16,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,0.0,1Z1FV6990396738961
3,55.01,36.86,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z4X459A4204799741
4,35.87,24.03,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z4X459A4210604322
5,37.18,24.91,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z4X459A4215211336
6,63.34,49.41,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z7EW7860230845078
7,14.91,8.66,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z84W64A0308207152
8,15.98,9.73,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z84W64A0325902547
9,18.28,12.03,0,0,0,-0.0,0,0,0.0,-0.0,...,0.0,-0.0,0,0,0,0,0,0,-0.0,1Z84W64A0333588659
